## Part I - 1.4 Data Analysis

In [3]:
"""Global functions and variables"""

import pymysql

def open_conn():
    """open the connection before each test case"""
    conn = pymysql.connect(user='public', password='ece656yelp',
                                   host='maindb.czbva1am4d4u.us-east-2.rds.amazonaws.com',
                                   database='yelp_db')
    return conn

def close_conn(conn):
    """close the connection after each test case"""
    conn.close()

def executeQuery(conn, query, commit=False):
    """ fetch result after query"""
    cursor = conn.cursor()
    query_num = query.count(";")
    if query_num > 1:
        for result in cursor.execute(query, params=None, multi=True):
            if result.with_rows:
                result = result.fetchall()
    else:
        cursor.execute(query)
        result = cursor.fetchall()
    # we commit the results only if we want the updates to Can't leave a review dated before account creationthe database
    # to persist.
    if commit:
        conn.commit()
    else:
        conn.rollback()
    # close the cursor used to execute the query
    cursor.close()
    return result

yelp_conn = open_conn()

Conditions for determining which reviews are SPAM:  
Accuracy with WHERE review_count = 1: 61%  
Accuracy with WHERE review_count = 1 AND review.useful = 0 AND review.funny = 0 AND review.cool = 0: 62%  
Expanding POS tags looked at to include verbs on top of adjective and adverbs: 63%  
Look at the top 100 frequently appearing words instead of 1000: 53%  
Look at the top 5000 frequently appearing words instead of 1000: 66%  
Reducing POS tags looked at just adjectives: 67%  
Remove non-english reviews and replacing - and / with spaces: 67%  
Add review.date -yelping_since = 0: 70%  
Change query to 'SELECT text FROM user join review on user.id=review.user_id where average_stars =  5 or average_stars = 1': 71%  
Change query to 'SELECT text FROM review JOIN user ON user.id=review.user_id WHERE review_count = 1 AND review.useful = 0 AND review.funny = 0 AND review.cool = 0 AND review.date - yelping_since and average_stars = 5 limit 1000': 76%  
Change query to 'SELECT text FROM review JOIN user ON user.id=review.user_id WHERE review_count = 1 AND review.useful = 0 AND review.funny = 0 AND review.cool = 0 AND review.date - yelping_since and average_stars = 1 limit 1000': 80%  
Change query to only include restaurants: 83%
Change query to only include hotels: 85%
Change query to only include restaurants: 85%

### Description  
Now that we're confident that the data is sufficiently cleaned as to not produce erroneous results from our analysis outside of those of interest, we can begin performing the actual analysis of trends in the data. The trend that was analyzed is the affect different attributes about users and their reviews have on the likelihood that their review is spam. These attributes include the type of business being reviewed, the ratings of the reviews left by users, the average_stars of the user and the review_count of the user. In order to determine if a given query returns reviews that are spam a machine learning classifier was used that implemented a bag of words model and applied a Naive Bayes and random forest classifier